In [17]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import networkx as nx
import funcy as fy
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

import usgoc.preprocessing.graph.wl2 as wl2
import usgoc.datasets.unsafe_go as dataset
import usgoc.models.gnn as gnn
import usgoc.utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
split_i = 0 # the evaluated split number (between 0-9)
in_enc = "wl1"

with utils.cache_env(use_cache=True):
  ds = dataset.load_dataset()
  splits = dataset.get_split_idxs(ds)

[dbg] Preprocessing CFG 100/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 200/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 300/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 400/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 500/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 600/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 700/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 800/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 900/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 1000/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 1100/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 1200/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 1300/1400 (mode=atomic_blocks).
[dbg] Preprocessing CFG 1400/1400 (mode=atomic_blocks).


In [80]:
import usgoc.datasets.unsafe_go as dataset

limit_id = "v127_d127_f127_p127_nlp"
mode = "atomic_blocks"
in_enc = "wl1"
batch_size_limit=1

with utils.cache_env(use_cache=True):
  dims, train_ds, val_ds, test_ds = dataset.get_encoded_dataset_slices(
      ds, in_enc, splits, split_i, limit_id=limit_id, mode=mode,
      batch_size_limit=batch_size_limit)
  train_ds = train_ds.cache()
  val_ds = val_ds.cache()
  train_slice, val_slice, test_slice = dataset.get_dataset_slices(
      ds, splits, split_i)

In [89]:
list(list(train_ds)[1][0]["X"][3].numpy())

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0

In [83]:
i = splits[0]["model_selection"][0]["train"][1]
graphs = ds[0]
g = graphs[splits[0]["model_selection"][0]["train"][1]]
utils.draw_graph(g, layout="dot")
i

429

In [86]:
code = g.nodes(data=True)[35]["code"]
code

'var kindTypes = map[reflect.Kind]Type{\n\treflect.Bool:          TypeOf(true),\n\treflect.Uint8:         TypeOf(uint8(0)),\n\treflect.Int8:          TypeOf(int8(0)),\n\treflect.Uint16:        TypeOf(uint16(0)),\n\treflect.Int16:         TypeOf(int16(0)),\n\treflect.Uint32:        TypeOf(uint32(0)),\n\treflect.Int32:         TypeOf(int32(0)),\n\treflect.Uint64:        TypeOf(uint64(0)),\n\treflect.Int64:         TypeOf(int64(0)),\n\treflect.Uint:          TypeOf(uint(0)),\n\treflect.Int:           TypeOf(int(0)),\n\treflect.Float32:       TypeOf(float32(0)),\n\treflect.Float64:       TypeOf(float64(0)),\n\treflect.Uintptr:       TypeOf(uintptr(0)),\n\treflect.String:        TypeOf(""),\n\treflect.UnsafePointer: TypeOf(unsafe.Pointer(nil)),\n}'

In [ ]:
from transformers import CodeGenTokenizer

checkpoint = "Salesforce/codegen-350M-multi"
tokenizer = CodeGenTokenizer.from_pretrained(checkpoint, cache_dir="/app/.cache")
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
tokenizer

In [ ]:
def get_block_snippets(g, snippets=None):
  if snippets is None:
    snippets = set()
  for node, data in g.nodes(data=True):
    if "code" in data:
      snippets.add(data["code"])
  return snippets

graphs = ds[0]
snippets = {g.source_code for g in graphs}

for g in graphs:
  get_block_snippets(g, snippets)

snippets = list(snippets)

import json

with open("/app/data/unsafe-go-dataset/snippets.json", "w") as f:
  json.dump(snippets, f)

len(snippets)

In [24]:
import pickle
import json

with open("/app/data/unsafe-go-dataset/embs.pickle", "rb") as f:
  embs = pickle.load(f)
  
embs = [e.reshape((-1, 1024)).mean(axis=0) for e in embs]

with open("/app/data/unsafe-go-dataset/snippets.json", "r") as f:
  snippets = json.load(f)

In [ ]:
emb_dict = fy.zipdict(snippets, embs)

with open("/app/data/unsafe-go-dataset/emb_dict.pickle", "wb") as f:
  pickle.dump(emb_dict, f)

In [87]:
emb_dict = dataset.embed_code_snippets(graphs)

emb_dict[code]

array([-0.667 ,  3.625 ,  7.082 , ..., -3.238 , -0.1455,  5.844 ],
      dtype=float16)